In [1]:
pip install faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.6 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [8]:
from google.colab import files

In [13]:
!cp '/content/gdrive/MyDrive/Colab Notebooks/nmt_utils.py' .

In [14]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
m=10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|██████████| 10000/10000 [00:00<00:00, 21542.43it/s]


In [17]:
dataset[:10]

[('9 may 1998', '1998-05-09'),
 ('10.11.19', '2019-11-10'),
 ('9/10/70', '1970-09-10'),
 ('saturday april 28 1990', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('08 jul 2008', '2008-07-08'),
 ('8 sep 1999', '1999-09-08'),
 ('thursday january 1 1981', '1981-01-01')]

In [18]:
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 '<pad>': 36,
 '<unk>': 35,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34}

In [19]:
machine_vocab

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [20]:
Tx=30
Ty=10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print('X shape', X.shape)
print('Y shape', Y.shape)
print('Xoh shape', Xoh.shape)
print('Yoh shape', Yoh.shape)

X shape (10000, 30)
Y shape (10000, 10)
Xoh shape (10000, 30, 37)
Yoh shape (10000, 10, 11)


In [21]:
index=0
print('Source date:', dataset[index][0])
print('Target date:', dataset[index][1])

print()
print('Source after preprocessing', X[index])
print('Target after preprocessing', Y[index])

print()
print('Source after preprocessing one-hot', Xoh[index])
print('Target after preprocessing one-hot', Yoh[index])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing one-hot [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing one-hot [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [22]:
repeator=RepeatVector(Tx)
concatenator=Concatenate(axis=-1)
densor1=Dense(10,activation='tanh')
densor2=Dense(1, activation='relu')
activator=Activation(softmax,name='attention_weights')
dotor=Dot(axes=1)

In [23]:
def one_step_attention(a, s_prev):
    
    s_prev=repeator(s_prev)
    concat=concatenator([a,s_prev])
    e=densor1(concat)
    energies=densor2(e)
    alphas=activator(energies)
    context=dotor([alphas,a])
    
    return context

In [24]:
n_a=32
n_s=64

post_activation_LSTM_cell = LSTM(n_s, return_state=True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [25]:
def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X=Input(shape=(Tx, human_vocab_size))
    s0=Input(shape=(n_s,), name='s0')
    c0=Input(shape=(n_s,), name='c0')
    s=s0
    c=c0
    
    outputs=[]
    
    a=Bidirectional(LSTM(n_a,return_sequences=True))(X)
    
    for t in range(Ty):
        context=one_step_attention(a,s)
        
        s,_,c=post_activation_LSTM_cell(context,initial_state=[s,c])
        out=output_layer(s)
        outputs.append(out)
        
    model = Model(inputs=[X,s0,c0],outputs=outputs)
    
    return model

In [26]:
model=modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 37)]     0           []                               
                                                                                                  
 s0 (InputLayer)                [(None, 64)]         0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 30, 64)       17920       ['input_1[0][0]']                
                                                                                                  
 repeat_vector (RepeatVector)   (None, 30, 64)       0           ['s0[0][0]',                     
                                                                  'lstm[0][0]',               

In [28]:
opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999,decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
s0=np.zeros((m,n_s))
c0=np.zeros((m,n_s))
outputs=list(Yoh.swapaxes(0,1))

In [30]:
model.fit([Xoh,s0,c0], outputs, epochs=5, batch_size=100)

Epoch 1/5
100/100 [==============================] - 23s 31ms/step - loss: 17.4551 - dense_2_loss: 1.2606 - dense_2_1_loss: 1.2260 - dense_2_2_loss: 1.9759 - dense_2_3_loss: 2.7099 - dense_2_4_loss: 0.8432 - dense_2_5_loss: 1.3488 - dense_2_6_loss: 2.6487 - dense_2_7_loss: 1.0748 - dense_2_8_loss: 1.7302 - dense_2_9_loss: 2.6371 - dense_2_accuracy: 0.4284 - dense_2_1_accuracy: 0.5337 - dense_2_2_accuracy: 0.2496 - dense_2_3_accuracy: 0.0722 - dense_2_4_accuracy: 0.9601 - dense_2_5_accuracy: 0.2779 - dense_2_6_accuracy: 0.0476 - dense_2_7_accuracy: 0.8556 - dense_2_8_accuracy: 0.2468 - dense_2_9_accuracy: 0.1089
Epoch 2/5
100/100 [==============================] - 3s 31ms/step - loss: 9.4930 - dense_2_loss: 0.4183 - dense_2_1_loss: 0.2911 - dense_2_2_loss: 1.1960 - dense_2_3_loss: 2.1720 - dense_2_4_loss: 0.0252 - dense_2_5_loss: 0.3056 - dense_2_6_loss: 1.8019 - dense_2_7_loss: 0.0275 - dense_2_8_loss: 1.1359 - dense_2_9_loss: 2.1194 - dense_2_accuracy: 0.8674 - dense_2_1_accuracy: 0.9

In [31]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
s00=np.zeros((1,n_s))
c00=np.zeros((1, n_s))

for example in EXAMPLES:
  source=string_to_int(example, Tx, human_vocab)
  source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
  source=np.swapaxes(source,0,1)
  source=np.expand_dims(source,axis=0)
  prediction=model.predict([source,s00,c00])
  prediction=np.argmax(prediction, axis=-1)
  output=[inv_machine_vocab[int(i)] for i in prediction]
  print("source ",example)
  print('target ',''.join(output))



source  3 May 1979
target  1997-05-03
source  5 April 09
target  2094-04-04
source  21th of August 2016
target  2016-08-22
source  Tue 10 Jul 2007
target  2007-07-10
source  Saturday May 9 2018
target  2018-05-09
source  March 3 2001
target  2000-03-33
source  March 3rd 2001
target  2001-03-33
source  1 March 2001
target  2000-03-10
